# Lab 2: Language models

Laboration 2 in EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/assignment-2/

Author: Jonatan Kronander

## Objectives
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment
* Optionally read a short article on the importance of corpora


## Collecting a corpus


#### 1. Collect a corpus of at least 750,000 words. Alternatively, you can retrieve a corpus of novels by Selma Lagerlöf from this URL: http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/Selma.txt .

In [1]:
from urllib.request import urlopen

text = urlopen("http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/Selma.txt").read() # Open file and read
strText = str(text,'utf-8') # From bytes to string

In [2]:
text_file = open("Selma.txt", "w")
text_file.write(strText)
text_file.close()

You will check the number of words using the Unix command wc -w. 

In [3]:
import subprocess
cmd_0 = 'wc -w Selma.txt'
print(cmd_0)

p = subprocess.Popen(cmd_0, stdout=subprocess.PIPE, shell=True)
out_0, err = p.communicate() 
print("Number of words in text:" + str(out_0,'utf-8'))

wc -w Selma.txt
Number of words in text:  965943 Selma.txt



#### 2. Run the concordance program (https://github.com/pnugues/ilppp/tree/master/programs/ch02/python) to print the lines containing a specific word, for instance Nils.

In [4]:
pattern = "Nils"
width = " 10"
cmd_1 = "python concord.py Selma.txt "+pattern+width
print(cmd_1)

p = subprocess.Popen(cmd_1, stdout=subprocess.PIPE, shell=True)
out_1, err = p.communicate() 
print("\n ----Lines containing " + pattern + " is listed below---- \n")
print(str(out_1,'utf-8'))

python concord.py Selma.txt Nils 10

 ----Lines containing Nils is listed below---- 

Nils Holgersso
os Holger Nilssons LV. A
tt! Se på Nils gåsapåg! 
tt! Se på Nils Holgersso
ll Holger Nilsson i Väst
Jag heter Nils Holgersso
det värt, Nils Holgersso
att du är Nils gåsapåg, 
r han var Nils gåsapåg, 
r och hur Nils gåsapåg h
å han var Nils gåsapåg? 
tiden, då Nils Holgersso
honom vad Nils Holgersso
 året, då Nils Holgersso
osta dem. Nils Holgersso
 sägas om Nils Holgersso
 där stod Nils Holgersso
de syn på Nils Holgersso
ässen och Nils Holgersso
. Men vad Nils Holgersso
 du hört, Nils gåsapåg, 
och om då Nils Holgersso
och om nu Nils Holgersso
g ut, att Nils Holgersso
nte ville Nils Holgersso
iden, när Nils Holgersso
honom. Om Nils Holgersso
gen hand, Nils Holgersso
rumle ner Nils Holgersso
o, jag är Nils Holgersso
tiden, då Nils Holgersso
Det står: Nils Holgersso
fär innan Nils Holgersso
föda, men Nils Holgersso
 året, då Nils Holgersso
ssen över Nils Holgersso
 dag fick Nils Holgersso

#### 3. Run the tokenization program (https://github.com/pnugues/ilppp/tree/master/programs/ch05/python) on your corpus and count the words using the Unix sort and uniq commands.

In [5]:
def toLowercase(matchobj):
    """
    Helper function
    """
    return matchobj.group(1).lower()

In [6]:
import regex as re

cmd_2 = "python tokenizer.py < Selma.txt"
print(cmd_2)

p = subprocess.Popen(cmd_2, stdout=subprocess.PIPE, shell=True)
out_2, err = p.communicate() 

stringOut = str(out_2,'utf-8') # Bytes --> String

cleanString = re.sub(r'[^A-Za-z ]', '', stringOut) # Remove all non characters (-space)
cleanString = re.sub(r'(\p{Lu})', toLowercase, cleanString) # Lowercase all characters
nlString = re.sub(r'[ ]', r'\n', cleanString) # New line instead of space

text_file = open("selmaCorpus.txt", "w")
text_file.write(nlString)
text_file.close()

selmaString = open('selmaCorpus.txt', 'r')

selmaStringData = selmaString.readlines()
selmaStringData = [ x.rstrip() for x in selmaStringData ]

python tokenizer.py < Selma.txt


In [7]:
# Count words with unix
cmd_3 = "sort selmaCorpus.txt | uniq"
print(cmd_3)

p = subprocess.Popen(cmd_3, stdout=subprocess.PIPE, shell=True)
out_3, err = p.communicate() 

stringOut = str(out_3,'utf-8')

text_file = open("cleanSelmaCorpus.txt", "w")
text_file.write(stringOut)
text_file.close()

selmaString = open('cleanSelmaCorpus.txt', 'r')
data = selmaString.readlines()

corpusList = [ x.rstrip() for x in data ]

sort selmaCorpus.txt | uniq


In [8]:
corpusList[0:50]

['',
 'a',
 'abborrar',
 'abborrarna',
 'abborre',
 'abborren',
 'abborrens',
 'abborrn',
 'abcbok',
 'abcdarierna',
 'abessinien',
 'abessinierna',
 'abessiniska',
 'abraham',
 'abrahams',
 'absalon',
 'absolut',
 'absoluta',
 'abstinens',
 'abu',
 'ach',
 'achmed',
 'ack',
 'ackja',
 'ackompanjemanget',
 'ackompanjera',
 'ackord',
 'acquilon',
 'acquilons',
 'adam',
 'ade',
 'adeline',
 'adelsbrev',
 'adelsfrknarna',
 'adelsgrd',
 'adelskapets',
 'adelsman',
 'adelsmn',
 'aderton',
 'adertonde',
 'adertonhundrafyra',
 'adertonhundratalet',
 'adertonhundratjugotalet',
 'adertonhundratjugutalet',
 'adertonhundratolv',
 'adertonhundratre',
 'adertonhundratrettiotalet',
 'adertonhundratta',
 'adertonriga',
 'adertonringen']

In [9]:
print("There are " + str(len(corpusList)) + " words in the corpus.")

There are 38663 words in the corpus.


## Normalizing a corpus


#### 1. Write a program to insert \< s> and \< /s> tags to delimit sentences. You can start from the tokenization and modify it. Use a simple heuristics such as: a sentence starts with a capital letter and ends with a period. Estimate roughly the accuracy of your program.


In [10]:
import regex as re

cmd_4 = "python tokenizer.py < Selma.txt"
print(cmd_4)

p = subprocess.Popen(cmd_4, stdout=subprocess.PIPE, shell=True)
out_4, err = p.communicate() 

stringOut = str(out_4,'utf-8') # Bytes --> String
cleanString = re.sub(r'[^A-ZÅÄÖa-zåäö \.?!]', '', stringOut) # Remove all non characters (-space)
nlString = re.sub(r'[ ]', r'\n', cleanString) # New line instead of space

text_file = open("normSelmaCorpus.txt", "w")
text_file.write(nlString)
text_file.close()

normselmaString = open('normSelmaCorpus.txt', 'r')

normData = normselmaString.readlines()
normData = [ x.rstrip() for x in normData ]

python tokenizer.py < Selma.txt


In [11]:
normData[:20]

['Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Selma',
 'Lagerlöf',
 'Innehåll',
 'Den',
 'kristna',
 'dagvisan',
 '',
 'Sveriges',
 'karta',
 'I',
 '.',
 'Pojken',
 '',
 'Tomten']

#### 2. Modify your program to remove the punctuation signs and set all the text in lower case letters.

In [13]:
delimitNormData = normData.copy()
added = 0

for i, word in enumerate(normData):
    upperCase = re.findall(r'^[A-Z]', word) # Remove all non characters (-space)

    if upperCase:
        delimitNormData[i + added] = re.sub(r'(\p{Lu})', toLowercase, word)
        delimitNormData.insert(i + added, "<s>")
        added = added + 1
  
    dot = re.findall(r'[\.\!\?]', word) # Remove all non characters (-space)
    if dot:
        delimitNormData.insert(i + added + 1, "</s>")
        del delimitNormData[i + added]


In [14]:
delimitNormData[:30]

['<s>',
 'nils',
 '<s>',
 'holgerssons',
 'underbara',
 'resa',
 'genom',
 '<s>',
 'sverige',
 '<s>',
 'selma',
 '<s>',
 'lagerlöf',
 '<s>',
 'innehåll',
 '<s>',
 'den',
 'kristna',
 'dagvisan',
 '',
 '<s>',
 'sveriges',
 'karta',
 '<s>',
 'i',
 '</s>',
 '<s>',
 'pojken',
 '',
 '<s>']

#### 3. The result should be a normalized text without punctuation signs where all the sentences are delimited with < s> and < /s> tags.

In [15]:
sentences = []
flag = 0 # 0 = sentence in ongoing, 1 = finished sentence
sentence = ""

for word in delimitNormData:
    if word == "<s>":
        continue
    if word == "</s>":
        flag = 1
        sentences.append("<s>" + sentence + "</s>")
        sentence = ""
        continue
    sentence = sentence + " " + word

#### 4. The five last lines of the text should look like this:

< s > hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse < /s >

< s > när prästen sa detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg < /s >

< s > prästens ord tycktes redan ha gått i uppfyllelse < /s >

< s > där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad < /s >

< s > hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare < /s >

In [16]:
sentences[len(sentences)-5:]

['<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste  och sådan kärlek måste vändas i välsignelse</s>',
 '<s> när prästen sa detta  kom alla människor att se bort mot klara gulla  och de förundrade sig över vad de såg</s>',
 '<s> prästens ord tycktes redan ha gått i uppfyllelse</s>',
 '<s> där stod klara fina gulleborg ifrån skrolycka  hon  som var uppkallad efter själva solen  vid sina föräldrars grav och lyste som en förklarad</s>',
 '<s> hon var likaså vacker som den söndagen  då hon gick till kyrkan i den röda klänningen  om inte vackrare</s>']

In [17]:
printOut = ""

for sentence in sentences:
    printOut = printOut + sentence + " "
    
text_file = open("selmaLow.txt", "w")
text_file.write(printOut)
text_file.close()

 #### Estimate roughly the accuracy of your program.

Ngra fel

In [18]:
dot = re.findall(r'[\.\!\?]', strText)

## Counting unigrams and bigrams

#### 1. Read and try programs to compute the frequency of unigrams and bigrams of the training set: [Program folder] (https://github.com/pnugues/ilppp/tree/master/programs/ch05/python)

In [20]:
cmd_5 = "python count.py < Selma.txt"
print(cmd_5)

p = subprocess.Popen(cmd_5, stdout=subprocess.PIPE, shell=True)
out_5, err = p.communicate() 

stringOut = str(out_5,'utf-8') # Bytes --> String
lines = len(re.findall(r'\n', stringOut))

print("There are " + str(lines) + " unigrams in training set.")
print("Most common words:\n" + stringOut[:105])

python count.py < Selma.txt
There are 39356 unigrams in training set.
Most common words:
och 	 37799
att 	 28914
han 	 22743
det 	 22087
i 	 17072
som 	 16790
hade 	 14955
på 	 14634
hon 	 14093


In [21]:
cmd_6 = "python count_bigrams.py < Selma.txt"
print(cmd_6)

p = subprocess.Popen(cmd_6, stdout=subprocess.PIPE, shell=True)
out_6, err = p.communicate() 

stringOut = str(out_6,'utf-8') # Bytes --> String
lines = len(re.findall(r'\n', stringOut))

print("There are " + str(lines) + " bigrams in training set.")
print("Some bigrams:\n" + stringOut[:214])

python count_bigrams.py < Selma.txt
There are 319877 bigrams in training set.
Some bigrams:
3 	 ('nils', 'holgerssons')
1 	 ('holgerssons', 'underbara')
1 	 ('underbara', 'resa')
4 	 ('resa', 'genom')
3 	 ('genom', 'sverige')
1 	 ('sverige', 'selma')
11 	 ('selma', 'lagerlöf')
2 	 ('lagerlöf', 'innehåll')


#### 2. What is the possible number of bigrams and their real number? 

As there are 965 943 words in the corpus the maximum possible number of bigrams is 965 943 * 2 - 2 = 1 931 884

Their real value for selma corpus is, as printed above, 319 877. 

#### Explain why such a difference. What would be the possible number of 4-grams.

This gives a difference of (1 931 884 - 319 877 = ) 1 612 007 "missing" bigrams. This is because of there is many bigrams that are occuring more than once. For example "selma lagerlöf" is occuring 11 times, as seen printed above. 

#### 3. Propose a solution to cope with bigrams unseen in the corpus. This topic will be discussed during the lab session.

backoff, använd unigram etc. /small value


## Computing the likelihood of a sentence


#### 1. Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [Program folder]. (https://github.com/pnugues/ilppp/tree/master/programs/ch05/python)


In [29]:
def string2dict(text):
    """
    Creates a dict with (word:number of apperences) from input string
    
    Input string, string
    Output dict
    """
    stringList = re.findall(r"<*\/*\w+>*",text)
    stringDict = {key : 0 for key in stringList}
    
    for m in re.finditer(r"<*\/*\w+>*", text): # Iterate thorugh every word
        s = m.start()
        e = m.end()
        word = text[s:e]
        stringDict[word] = stringDict[word] + 1
        
    return stringDict

In [119]:
text = open('selmaLow.txt').read()  

dictWord = string2dict(text)

In [45]:
def probModelUnigram(words, sentence):
    """
    Returns dict with prob of sentence
    
    :param dict:
    :param string:
    :return dict: (values = [wi, C(wi), #words, P(wi)])
    """
    sentence = re.sub(r'(\p{Lu})', toLowercase, sentence)

    
    listSentence = re.findall(r"<*\/*\w+>*",sentence)
        
    #uniModel = {key : list([]) for key in listSentence}
    uniModel = []
    nbrWords = 1086836
    
    for word in listSentence:
        if word in words:
            uniModel.append([word , words[word], nbrWords, words[word]/nbrWords])
        else:
            uniModel.append([word , 0, nbrWords, 0])
        
    return uniModel

In [51]:
uniList = probModelUnigram(dictWord, "det var en gång en katt som hette nils </s>")

In [64]:
print("[wi, C(wi), #words, P(wi)] : ")
for element in uniList:
    print(element)

[wi, C(wi), #words, P(wi)] : 
['det', 22087, 1086836, 0.020322293335885082]
['var', 12852, 1086836, 0.011825151172762036]
['en', 13921, 1086836, 0.012808740233117047]
['gång', 1332, 1086836, 0.0012255758918548888]
['en', 13921, 1086836, 0.012808740233117047]
['katt', 15, 1086836, 1.3801530313681181e-05]
['som', 16790, 1086836, 0.015448512931113802]
['hette', 107, 1086836, 9.845091623759242e-05]
['nils', 84, 1086836, 7.728856975661462e-05]
['</s>', 63698, 1086836, 0.05860865852805759]


In [123]:
import math
from scipy import stats

def totModel(probList):
    """
    THIS IS NOT WORKING PROBERLY. WHAT IS THE DEFINITION OF ALL THE OUTPUTS?!
    
    Prob. bigrams, Geometric mean prob., Entropy rate, Perplexity
    
    :param dict:
    :return list():
    """
    sentenceProb = 1
    probSum = []
    entropy = 0
    
    for words in probList:
        sentenceProb *= words[-1] # Prob. bigrams
        probSum.append(words[-1]) # Geometric mean prob
    
    if sentenceProb == 0:
        entropy = 0
    else:
        entropy = - math.log2(sentenceProb) / len(probList) # Entropy
    
    perplexity = 2**entropy #perplexity
    
    return [sentenceProb, stats.gmean(probSum), entropy, perplexity] #entropy*len(probDict.keys()), perplexity]

In [124]:
totModelList = totModel(uniList)
print(totModelList)

[4.594552317604714e-27, 0.0023239368373240534, 8.749213426641543, 430.3042939632865]


#### 2. Write a program to compute the sentence probability using bigrams.

In [109]:
def count_bigrams(words):
    bigrams = [tuple(words[inx:inx + 2]) for inx in range(len(words) - 1)]
    frequencies = {}
    for bigram in bigrams:
        if bigram in frequencies:
            frequencies[bigram] += 1
        else:
            frequencies[bigram] = 1
    return frequencies

In [131]:
def probModelBigram(text, sentence, totwords = dictWord):
    """
    Returns dict with prob of sentence bigrams
    
    :param text:
    :param string:
    :return dict: (values = [wi wi+1 Ci,i+1 C(i) P(wi+1|wi)])
    """
    
    words = re.findall("<*\/*\p{L}+>*", sentence)
    bigrams = [tuple(words[inx:inx + 2]) for inx in range(len(words) - 1)]
    
    wordsInText = re.findall("<*\/*\w+>*", text)
    freq = count_bigrams(wordsInText)
    
    biModel = []
    nbrWords = 1086836

    for bigram in bigrams:
        firstWord = re.findall(bigram[0], text)
        if bigram in freq:
            biModel.append([bigram[0], bigram[1], freq[bigram], len(firstWord) , freq[bigram]/len(firstWord)])
        else:
            if bigram[1] in totwords:
                biModel.append([bigram[0], bigram[1], 0, len(firstWord), totwords[bigram[1]]/nbrWords]) # Last element *backoff:
            else:
                biModel.append([bigram[0], bigram[1], 0, len(firstWord), 1*10**(-25)]) # Very small prob.

  
    return biModel

In [132]:
text = open('selmaLow.txt').read()  

biList = probModelBigram(text, "<s> det var en gång en katt som hette nils </s>")

In [133]:
print("[wi wi+1 Ci,i+1 C(i) P(wi+1|wi)] :")
for element in biList:
    print(element)

[wi wi+1 Ci,i+1 C(i) P(wi+1|wi)] :
['<s>', 'det', 5754, 63698, 0.09033250651511822]
['det', 'var', 4023, 26764, 0.15031385443132567]
['var', 'en', 753, 22780, 0.03305531167690957]
['en', 'gång', 695, 94886, 0.0073245789684463465]
['gång', 'en', 23, 2194, 0.010483135824977211]
['en', 'katt', 5, 94886, 5.269481272263558e-05]
['katt', 'som', 2, 488, 0.004098360655737705]
['som', 'hette', 50, 18172, 0.00275148580233326]
['hette', 'nils', 0, 109, 7.728856975661462e-05]
['nils', '</s>', 2, 121, 0.01652892561983471]


In [134]:
totModelList = totModel(biList)
print(totModelList)

[2.6161546826867327e-23, 0.0055178049169432473, 7.5016898338305875, 181.23148879898764]


#### 3. Select five sentences in your test set and run your programs on them.

In [135]:
uniList1 = probModelUnigram(dictWord, "HaDDwad jaawd awDDacc awger </s>")
totUniList1 = totModel(uniList1) 
uniList2 = probModelUnigram(dictWord, "Då var det genast, som om det hade funnits ett hemligt band eller samförstånd mellan oss </s>")
totUniList2 = totModel(uniList2) 
uniList3 = probModelUnigram(dictWord, "Hon ryckte till </s>")
totUniList3 = totModel(uniList3) 
uniList4 = probModelUnigram(dictWord, "Idag, är: det€£ måndag </s>")
totUniList4 = totModel(uniList4) 
uniList5 = probModelUnigram(dictWord, "det var en gång en katt som hette nils </s>")
totUniList5 = totModel(uniList5) 

/anaconda3/envs/py36/lib/python3.6/site-packages/scipy/stats/stats.py:306: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


In [136]:
biList1 = probModelBigram(text, "<s> HaDDwad jaawd awDDacc awger </s>")
totBiList1 = totModel(biList1) 
biList2 = probModelBigram(text, "<s> Då var det genast, som om det hade funnits ett hemligt band eller samförstånd mellan oss </s>")
totBiList2 = totModel(biList2) 
biList3 = probModelBigram(text, "<s> Hon ryckte till </s>")
totBiList3 = totModel(biList3) 
biList4 = probModelBigram(text, "<s> Idag är det måndag </s>")
totBiList4 = totModel(biList4) 
biList5 = probModelBigram(text, "<s> det var en gång en katt som hette nils </s>")
totBiList5 = totModel(biList5) 

#### 4. Tabulate your results as in the examples below with the sentence "Det var en gång en katt som hette Nils":


In [151]:
print("Unigram model")
print("=====================================================")
print("wi C(wi) #words P(wi)")
print("=====================================================")
for element in uniList1:
    print(element)        
print("=====================================================")

print("Prob. unigrams: ", totUniList1[0])
print("Geometric mean prob.: ", totUniList1[1])
print("Entropy rate: ", totUniList1[2])
print("Perplexity: ", totUniList1[3])

print("=====================================================")
print("=====================================================")

print("Bigram model")
print("=====================================================")
print("wi wi+1 Ci,i+1 C(i) P(wi+1|wi)")
print("=====================================================")
for element in biList1:
    print(element)        
print("=====================================================")
print("Prob. unigrams: ", totBiList1[0])
print("Geometric mean prob.: ", totBiList1[1])
print("Entropy rate: ", totBiList1[2])
print("Perplexity: ", totBiList1[3]) 

Unigram model
wi C(wi) #words P(wi)
['haddwad', 0, 1086836, 0]
['jaawd', 0, 1086836, 0]
['awddacc', 0, 1086836, 0]
['awger', 0, 1086836, 0]
['</s>', 63698, 1086836, 0.05860865852805759]
Prob. unigrams:  0.0
Geometric mean prob.:  0.0
Entropy rate:  0
Perplexity:  1
Bigram model
wi wi+1 Ci,i+1 C(i) P(wi+1|wi)
['<s>', 'HaDDwad', 0, 63698, 1e-25]
['HaDDwad', 'jaawd', 0, 0, 1e-25]
['jaawd', 'awDDacc', 0, 0, 1e-25]
['awDDacc', 'awger', 0, 0, 1e-25]
['awger', '</s>', 0, 0, 0.05860865852805759]
Prob. unigrams:  5.860865852805761e-102
Geometric mean prob.:  5.67012138399e-21
Entropy rate:  67.25711037242209
Perplexity:  1.763630674333816e+20


In [152]:
print("Unigram model")
print("=====================================================")
print("wi C(wi) #words P(wi)")
print("=====================================================")
for element in uniList2:
    print(element)        
print("=====================================================")

print("Prob. unigrams: ", totUniList2[0])
print("Geometric mean prob.: ", totUniList2[1])
print("Entropy rate: ", totUniList2[2])
print("Perplexity: ", totUniList2[3])

print("=====================================================")
print("=====================================================")

print("Bigram model")
print("=====================================================")
print("wi wi+1 Ci,i+1 C(i) P(wi+1|wi)")
print("=====================================================")
for element in biList2:
    print(element)        
print("=====================================================")
print("Prob. unigrams: ", totBiList2[0])
print("Geometric mean prob.: ", totBiList2[1])
print("Entropy rate: ", totBiList2[2])
print("Perplexity: ", totBiList2[3]) 

Unigram model
wi C(wi) #words P(wi)
['då', 4690, 1086836, 0.004315278478077649]
['var', 12852, 1086836, 0.011825151172762036]
['det', 22087, 1086836, 0.020322293335885082]
['genast', 527, 1086836, 0.0004848937650206655]
['som', 16790, 1086836, 0.015448512931113802]
['om', 8490, 1086836, 0.007811666157543549]
['det', 22087, 1086836, 0.020322293335885082]
['hade', 14955, 1086836, 0.013760125722740137]
['funnits', 72, 1086836, 6.624734550566967e-05]
['ett', 5224, 1086836, 0.004806612957244699]
['hemligt', 15, 1086836, 1.3801530313681181e-05]
['band', 55, 1086836, 5.060561115016433e-05]
['eller', 1360, 1086836, 0.001251338748440427]
['samförstånd', 2, 1086836, 1.8402040418241575e-06]
['mellan', 465, 1086836, 0.00042784743972411663]
['oss', 990, 1086836, 0.000910901000702958]
['</s>', 63698, 1086836, 0.05860865852805759]
Prob. unigrams:  1.9849823074840453e-49
Geometric mean prob.:  0.00136509273484
Entropy rate:  9.516785323592003
Perplexity:  732.5509648383886
Bigram model
wi wi+1 Ci,i+1 

In [155]:
print("Unigram model")
print("=====================================================")
print("wi C(wi) #words P(wi)")
print("=====================================================")
for element in uniList3:
    print(element)        
print("=====================================================")
print("Prob. unigrams: ", totUniList3[0])
print("Geometric mean prob.: ", totUniList3[1])
print("Entropy rate: ", totUniList3[2])
print("Perplexity: ", totUniList3[3])

print("=====================================================")
print("=====================================================")

print("Bigram model")
print("=====================================================")
print("wi wi+1 Ci,i+1 C(i) P(wi+1|wi)")
print("=====================================================")
for element in biList3:
    print(element)        
print("=====================================================")
print("Prob. unigrams: ", totBiList3[0])
print("Geometric mean prob.: ", totBiList3[1])
print("Entropy rate: ", totBiList3[2])
print("Perplexity: ", totBiList3[3]) 

Unigram model
wi C(wi) #words P(wi)
['hon', 14093, 1086836, 0.012966997780713925]
['ryckte', 119, 1086836, 0.00010949214048853736]
['till', 9445, 1086836, 0.008690363587514583]
['</s>', 63698, 1086836, 0.05860865852805759]
Prob. unigrams:  7.231395429779202e-10
Geometric mean prob.:  0.00518567777864
Entropy rate:  7.591251720121058
Perplexity:  192.83882313680908
Bigram model
wi wi+1 Ci,i+1 C(i) P(wi+1|wi)
['<s>', 'Hon', 0, 63698, 1e-25]
['Hon', 'ryckte', 0, 0, 0.00010949214048853736]
['ryckte', 'till', 36, 188, 0.19148936170212766]
['till', '</s>', 262, 12110, 0.021635012386457472]
Prob. unigrams:  4.5361222002560064e-32
Geometric mean prob.:  1.45938939631e-08
Entropy rate:  26.030059882575564
Perplexity:  68521807.99213421


In [156]:
print("Unigram model")
print("=====================================================")
print("wi C(wi) #words P(wi)")
print("=====================================================")
for element in uniList4:
    print(element)        
print("=====================================================")
print("Prob. unigrams: ", totUniList4[0])
print("Geometric mean prob.: ", totUniList4[1])
print("Entropy rate: ", totUniList4[2])
print("Perplexity: ", totUniList4[3])

print("=====================================================")
print("=====================================================")

print("Bigram model")
print("=====================================================")
print("wi wi+1 Ci,i+1 C(i) P(wi+1|wi)")
print("=====================================================")
for element in biList4:
    print(element)        
print("=====================================================")
print("Prob. unigrams: ", totBiList4[0])
print("Geometric mean prob.: ", totBiList4[1])
print("Entropy rate: ", totBiList4[2])
print("Perplexity: ", totBiList4[3]) 

Unigram model
wi C(wi) #words P(wi)
['idag', 4, 1086836, 3.680408083648315e-06]
['är', 6378, 1086836, 0.005868410689377239]
['det', 22087, 1086836, 0.020322293335885082]
['måndag', 1, 1086836, 9.201020209120787e-07]
['</s>', 63698, 1086836, 0.05860865852805759]
Prob. unigrams:  2.3669384074492462e-17
Geometric mean prob.:  0.000472974134731
Entropy rate:  11.0459510896766
Perplexity:  2114.2805210048605
Bigram model
wi wi+1 Ci,i+1 C(i) P(wi+1|wi)
['<s>', 'Idag', 0, 63698, 1e-25]
['Idag', 'är', 0, 0, 0.005868410689377239]
['är', 'det', 584, 29703, 0.019661313672019662]
['det', 'måndag', 0, 26764, 9.201020209120787e-07]
['måndag', '</s>', 0, 7, 0.05860865852805759]
Prob. unigrams:  6.2220113221013024e-37
Geometric mean prob.:  5.73833844795e-08
Entropy rate:  24.054791698141408
Perplexity:  17426647.261584166


In [148]:
print("Unigram model")
print("=====================================================")
print("wi C(wi) #words P(wi)")
print("=====================================================")
for element in uniList5:
    print(element)        
print("=====================================================")
print("Prob. unigrams: ", totUniList5[0])
print("Geometric mean prob.: ", totUniList5[1])
print("Entropy rate: ", totUniList5[2])
print("Perplexity: ", totUniList5[3])

Unigram model
wi C(wi) #words P(wi)
['det', 22087, 1086836, 0.020322293335885082]
['var', 12852, 1086836, 0.011825151172762036]
['en', 13921, 1086836, 0.012808740233117047]
['gång', 1332, 1086836, 0.0012255758918548888]
['en', 13921, 1086836, 0.012808740233117047]
['katt', 15, 1086836, 1.3801530313681181e-05]
['som', 16790, 1086836, 0.015448512931113802]
['hette', 107, 1086836, 9.845091623759242e-05]
['nils', 84, 1086836, 7.728856975661462e-05]
['</s>', 63698, 1086836, 0.05860865852805759]
Prob. unigrams:  4.594552317604714e-27
Geometric mean prob.:  0.00232393683732
Entropy rate:  8.749213426641543
Perplexity:  430.3042939632865


Bigram model
wi wi+1 Ci,i+1 C(i) P(wi+1|wi)
['<s>', 'HaDDwad', 0, 63698, 1e-25]
['HaDDwad', 'jaawd', 0, 0, 1e-25]
['jaawd', 'awDDacc', 0, 0, 1e-25]
['awDDacc', 'awger', 0, 0, 1e-25]
['awger', '</s>', 0, 0, 0.05860865852805759]
Prob. unigrams:  5.860865852805761e-102
Geometric mean prob.:  5.67012138399e-21
Entropy rate:  67.25711037242209
Perplexity:  1.763630674333816e+20


In [142]:



=====================================================

=====================================================
<s> det 5913 62283 0.09493762342854391
det var 4023 22086 0.1821515892420538
var en 753 12852 0.05859010270774977
en gång 695 13921 0.04992457438402414
gång en 23 1332 0.017267267267267267
en katt 5 13921 0.0003591695998850657
katt som 2 15 0.13333333333333333
som hette 50 16790 0.0029779630732578916
hette nils 0 107 0.0 *backoff: 7.728856975661462e-05
nils </s> 2 84 0.023809523809523808
=====================================================
Prob. bigrams: 2.292224542392586e-19
Geometric mean prob.: 0.013678098151101147
Entropy rate: 6.191988542790593
Perplexity: 73.10957919390972""")

Unigram model
wi C(wi) #words P(wi)
det 22086 1086836 0.02032137323386417
var 12852 1086836 0.011825151172762036
en 13921 1086836 0.012808740233117047
gång 1332 1086836 0.0012255758918548888
en 13921 1086836 0.012808740233117047
katt 15 1086836 1.3801530313681181e-05
som 16790 1086836 0.015448512931113802
hette 107 1086836 9.845091623759242e-05
nils 84 1086836 7.728856975661462e-05
</s> 62283 1086836 0.057306714168467
Prob. unigrams:   4.4922846219128876e-27
Geometric mean prob.: 0.0023187115559242404
Entropy rate:   8.752460922513437
Perplexity:   431.2739967353978


Bigram model
wi wi+1 Ci,i+1 C(i) P(wi+1|wi)
<s> det 5913 62283 0.09493762342854391
det var 4023 22086 0.1821515892420538
var en 753 12852 0.05859010270774977
en gång 695 13921 0.04992457438402414
gång en 23 1332 0.017267267267267267
en katt 5 13921 0.0003591695998850657
katt som 2 15 0.13333333333333333
som hette 50 16790 0.0029779630732578916
hette nils 0 107 0.0 *backoff: 7.728856975661462e-05
nils </s> 2 84 0.023809523

## Reading

As an application of n-grams, execute the Jupyter notebook by Peter Norvig here (http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb). Just run all the cells and be sure that you understand the code. You will find the data here (http://norvig.com/ngrams/). In you report, you will describe one experiment with a long string of words your will create yourself.

# Complement

As a complement, you can read a paper by Church (https://researcher.watson.ibm.com/researcher/view.php?person=us-kwchurch) and Hanks, Word Association Norms, Mutual Information, and Lexicography, (https://www.aclweb.org/anthology/J90-1003) Computational Linguistics, 16(1):22-29, 1990, as well as another one on backoff by Brants et al. (2007) Large language models in machine translation (https://www.aclweb.org/anthology/D07-1090).
